In [2]:
import pandas as pd
import numpy as np
from collections import deque
from tqdm import tqdm
import sqlite3

In [3]:
art_path = "C:/Users/egor1/Desktop/Work/RQC/PZM/data/input/ART.xlsx"
cade_path = "C:/Users/egor1/Desktop/Work/RQC/PZM/data/input/CADE.xlsx"
gamm_path = "C:/Users/egor1/Desktop/Work/RQC/PZM/data/input/GAMM.xlsx"
links_path = "C:/Users/egor1/Desktop/Work/RQC/PZM/data/input/LINKS.xlsx"
mach_path = "C:/Users/egor1/Desktop/Work/RQC/PZM/data/input/MACH.xlsx"
maint_path = "C:/Users/egor1/Desktop/Work/RQC/PZM/data/input/MAINT.xlsx"
pick_path = "C:/Users/egor1/Desktop/Work/RQC/PZM/data/input/PICKLIST.xlsx"
orders_path = "C:/Users/egor1/Desktop/Work/RQC/PZM/data/input/PZM_1C_ORDERS.xlsx"
serie_path = "C:/Users/egor1/Desktop/Work/RQC/PZM/data/input/PZM_SERIE.xlsx"

In [4]:
df_art = pd.read_excel(art_path, engine='openpyxl')
df_cade = pd.read_excel(cade_path, engine='openpyxl')
df_gamm = pd.read_excel(gamm_path, engine='openpyxl')
df_links = pd.read_excel(links_path, engine='openpyxl')
df_mach = pd.read_excel(mach_path, engine='openpyxl')
df_maint = pd.read_excel(maint_path, engine='openpyxl')
df_pick = pd.read_excel(pick_path, engine='openpyxl')
df_orders = pd.read_excel(orders_path, engine='openpyxl')
df_serie = pd.read_excel(serie_path, engine='openpyxl')

KeyboardInterrupt: 

In [ ]:
class Operation:
    def __init__(self):
        self.op_id = None  # глобальный id операции
        self.order_id = None  # Номер заказа
        self.phase = None  # id операции в технологическом маршруте
        self.route = None  # Технологический маршрут
        self.duration = None  # Продолжительность операции
        
        self.predecessors = np.empty(shape=(0,))  # Список предшествующих операций
        self.successors = np.empty(shape=(0,))  # Список последующих операций
        self.resources = np.empty(shape=(0,))  # Требования по ресурсам

        self.early_start = 0
        self.early_finish = 0
        self.late_start = None
        self.late_finish = None
        self.is_critical = False

        self.machine = None 

    def set_data(self, op_id, order_id, phase, route, duration, predecessors, successors, resources):
        self.op_id = op_id
        self.order_id = order_id
        self.phase = phase
        self.route = route
        self.duration = duration

        self.predecessors = predecessors
        self.successors = successors
        self.resources = resources
        
    def __repr__(self):
        return (f"\n\n Operation(op_id={self.op_id},\n\t order_id={self.order_id},\n\t phase_id={self.phase},\n\t route={self.route},\n\t "
                f"duration={self.duration},\n\t predecessors={self.predecessors},\n\t successors={self.successors},\n\t "
                f"resources={self.resources},\n\t machine={self.machine},\n\t early_start={self.early_start},\n\t early_finish={self.early_finish},\n\t "
                f"late_start={self.late_start},\n\t late_finish={self.late_finish},\n\t is_critical={self.is_critical})")


In [ ]:
class OperationManager:
    def __init__(self):
        self.operations = {}  # Словарь для хранения операций по их ID

    def add_operation(self, op_id, order_id, phase, route, duration, predecessors, successors, resources):
        operation = Operation()
        
        operation.set_data(op_id, order_id, phase, route, duration, predecessors, successors, resources)
        self.operations[op_id] = operation

    def get_operation(self, op_id):
        return self.operations.get(op_id)

    def save_to_csv(self, filename):
        data = []
        for operation in self.operations.values():
            data.append({
                'op_id': operation.op_id,
                'order_id': operation.order_id,
                'phase': operation.phase,
                'route': operation.route,
                'duration': operation.duration,
                'machine': operation.machine,
                'early_start': operation.early_start,
                'early_finish': operation.early_finish,
                'late_start': operation.late_start,
                'late_finish': operation.late_finish,
                'is_critical': operation.is_critical
            })
        df = pd.DataFrame(data)
        df.to_csv(filename, index=False)

    def save_to_db(self, db_filename):
        conn = sqlite3.connect(db_filename)
        c = conn.cursor()
        
        c.execute('''CREATE TABLE IF NOT EXISTS operations (
                        op_id TEXT PRIMARY KEY,
                        order_id TEXT,
                        phase TEXT,
                        route TEXT,
                        duration INTEGER,
                        predecessors TEXT,
                        successors TEXT,
                        resources TEXT,
                        early_start INTEGER,
                        early_finish INTEGER,
                        late_start INTEGER,
                        late_finish INTEGER,
                        is_critical BOOLEAN,
                        machine TEXT)''')
        
        for operation in self.operations.values():
            c.execute('''INSERT OR REPLACE INTO operations (
                            op_id, order_id, phase, route, duration, predecessors, successors, resources,
                            early_start, early_finish, late_start, late_finish, is_critical, machine)
                         VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', 
                      (operation.op_id, operation.order_id, operation.phase, operation.route, operation.duration,
                       ','.join(map(str, operation.predecessors)), ','.join(map(str, operation.successors)),
                       ','.join(map(str, operation.resources)), operation.early_start, operation.early_finish,
                       operation.late_start, operation.late_finish, operation.is_critical, operation.machine))
        
        conn.commit()
        conn.close()

    def load_from_db(self, db_filename):
        conn = sqlite3.connect(db_filename)
        c = conn.cursor()
        
        c.execute('SELECT * FROM operations')
        rows = c.fetchall()
        
        for row in rows:
            op_id, order_id, phase, route, duration, predecessors, successors, resources, \
            early_start, early_finish, late_start, late_finish, is_critical, machine = row
            
            operation = Operation()
            operation.set_data(op_id, order_id, phase, route, duration,
                               predecessors.split(','), successors.split(','), resources.split(','))
            operation.early_start = early_start
            operation.early_finish = early_finish
            operation.late_start = late_start
            operation.late_finish = late_finish
            operation.is_critical = bool(is_critical)
            operation.machine = machine
            
            self.operations[op_id] = operation
        
        conn.close()

In [ ]:
def compute_critical_path(manager):
    # Инициализация ранних стартов и финишей
    for operation in manager.operations.values():
        operation.early_start = 0
        operation.early_finish = 0

    # Прямой проход (Forward Pass)
    queue = deque([op_id for op_id, op in manager.operations.items() if len(op.predecessors) == 0])
    while queue:
        op_id = queue.popleft()
        operation = manager.get_operation(op_id)
        operation.early_finish = operation.early_start + operation.duration
        for succ_id in operation.successors:
            successor = manager.get_operation(succ_id)
            successor.early_start = max(successor.early_start, operation.early_finish)
            queue.append(succ_id)

    # Инициализация поздних стартов и финишей
    max_early_finish = max([op.early_finish for op in manager.operations.values()])
    for operation in manager.operations.values():
        operation.late_start = max_early_finish
        operation.late_finish = max_early_finish

    # Обратный проход (Backward Pass)
    queue = deque([op_id for op_id, op in manager.operations.items() if len(op.successors) == 0])
    while queue:
        op_id = queue.popleft()
        operation = manager.get_operation(op_id)
        operation.late_start = operation.late_finish - operation.duration
        for pred_id in operation.predecessors:
            predecessor = manager.get_operation(pred_id)
            predecessor.late_finish = min(predecessor.late_finish, operation.late_start)
            queue.append(pred_id)

    # Определение критического пути
    critical_path = []
    for op_id, operation in manager.operations.items():
        if operation.early_start == operation.late_start:
            operation.is_critical = True
            critical_path.append(op_id)

    return critical_path

In [ ]:
ope_manager = OperationManager()

for order in tqdm(df_orders.itertuples(index=True, name='Pandas')):
    order_id = order.NOF
    order_artic = order.CODEARTIC
    order_artic_ver = order.VERSION
    order_quantity = int(order.QTE)

    order_art_info = df_art.loc[(df_art['CODEARTIC'] == order_artic) & (df_art['VER_CODEARTIC'] == order_artic_ver)]

    if len(order_art_info) > 1:
        print('Error art_info', order_id)

    order_route  = order_art_info.iloc[0]['NOMG']
    order_operations = df_gamm.loc[df_gamm['NOMG'] == order_route].sort_values(by='START_PHASE')

    for operation in order_operations.itertuples(index=True, name='Pandas'):
        operation_duration = int(operation.RUC*order_quantity*60)
        operation_phase = int(operation.START_PHASE)
        
        operation_successors=np.empty(shape=(0,))

        op_id = order_route + '/_/' + str(operation_phase)

        if not pd.isna(operation.END_PHASE):
            operation_phase_next = int(operation.END_PHASE)
            op_id_next = order_route + '/_/' + str(operation_phase_next)
            operation_successors = np.append(operation_successors, op_id_next)

            if ope_manager.get_operation(op_id_next) is None:
                ope_manager.add_operation(op_id=op_id_next, order_id=order_id, phase=operation_phase, route=order_route, duration=None, 
                    predecessors=np.array([op_id]), successors=np.empty(shape=(0,)), resources=np.empty(shape=(0,)))
            else:
                operation = ope_manager.get_operation(op_id_next)
                operation.predecessors = np.append(operation.predecessors, op_id)
            

        operation_suc = df_links.loc[(df_links['B_O_NOF'] == order_route) & (df_links['B_O_NOPHASE'] == operation_phase)]

        for suc in operation_suc.itertuples(index=True, name='Pandas'):
            suc_op_order = suc.NOF
            suc_op_phase = int(suc.NOPHASE)
            
            suc_id = suc_op_order + '/_/' + str(suc_op_phase)

            operation_successors = np.append(operation_successors, suc_id)

            if ope_manager.get_operation(suc_id) is None:
                ope_manager.add_operation(op_id=suc_id, order_id=suc_op_order, phase=suc_op_phase, route=suc_op_order, duration=None, 
                    predecessors=np.array([op_id]), successors=np.empty(shape=(0,)), resources=np.empty(shape=(0,)))
            else:
                operation = ope_manager.get_operation(suc_id)
                operation.predecessors = np.append(operation.predecessors, op_id)
       

        if ope_manager.get_operation(op_id) is None:
            ope_manager.add_operation(op_id=op_id, order_id=order_id, phase=operation_phase, route=order_route, duration=operation_duration, 
                predecessors=np.empty(shape=(0,)), successors=operation_successors, resources=np.empty(shape=(0,)))
        else:
            operation = ope_manager.get_operation(op_id)
            operation.successors = np.append(operation.successors, operation_successors)
            operation.duration=operation_duration


11683it [07:53, 24.67it/s]


In [ ]:
ope_manager.save_to_db('operations.db')

In [ ]:
op_manager = OperationManager()
op_manager.load_from_db('operations.db')

In [ ]:
pzm_critical_path = compute_critical_path(ope_manager)
print("Критический путь:", pzm_critical_path)

Критический путь: ['C000195588/_/100020', 'C000195588/_/100010', 'C000195588/_/100030', 'C000195588/_/200010', 'C000195588/_/300010', 'C000195588/_/300020', 'C000195588/_/300030', 'C000195588/_/300040', 'C000195588/_/300050', 'C000195588/_/300060', 'C000195588/_/300070', 'C000195588/_/300080', 'C000195588/_/300090', 'C000195588/_/300100', 'C000195588/_/300110', 'C000195588/_/300120', 'C000195588/_/300130', 'C000195588/_/300140', 'C000195588/_/400010', 'C000195588/_/400020', 'C000195588/_/400030', 'C000195588/_/400040', 'C000195588/_/400050', 'C000195588/_/400060', 'C000195588/_/400070', 'C000195588/_/500010', 'C000195588/_/500020', 'C000195588/_/500030', 'C000195588/_/500040', 'C000195588/_/500050', 'C000195588/_/500060', 'C000195588/_/500070', 'C000195588/_/500080', 'C000195588/_/500090', 'C000195588/_/500100', 'C000195588/_/500110', 'C000195588/_/500120', 'C000195588/_/500130', 'C000195588/_/500140', 'C000195588/_/500150', 'C000195588/_/500160', 'C000195588/_/500170', 'C000195588/_/5

In [ ]:
ope_manager.save_to_csv('pzm_cpm.csv')

In [ ]:
ope_manager.save_to_db('operations_rcpm.db')

In [ ]:
# machines = df_mach.groupby('ILOT').apply(lambda x: [[machine, 0] for machine in x['MACHINE']]).to_dict()
machines = df_mach.groupby('ILOT').apply(
        lambda x: {machine: 0 for machine in x['MACHINE']}
    ).to_dict()

C:\Users\egor1\AppData\Local\Temp\ipykernel_17492\1729912097.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  machines = df_mach.groupby('ILOT').apply(


In [ ]:
for operation in tqdm(sorted(ope_manager.operations.values(), key=lambda op: op.early_start)):

    if operation.machine == None:
        resources = df_cade.loc[(df_cade['NOMG'] == operation.order_id) & (df_cade['START_PHASE'] == operation.phase), 'ILOT'].tolist()
        operation.resources = resources

    else :
        resources = operation.resources 

    min_release_time = float('inf') 
    min_machine = None
    min_recource = None

    for recource in resources:
        if recource in machines:
            for machine, release_time in machines[recource].items():
                if release_time < min_release_time:
                    min_release_time = release_time
                    min_machine = machine
                    min_recource = recource

    operation_start_time = max(min_release_time, operation.early_start)
    machines[min_recource][min_machine] = operation_start_time + operation.duration

    delta = operation_start_time - operation.early_start

    operation.early_start += delta
    operation.early_finish += delta
    operation.late_start += delta
    operation.late_finish += delta

    operation.machine = min_machine

        

100%|██████████| 272612/272612 [51:29<00:00, 88.23it/s]   


In [ ]:
ope_manager.save_to_db('operations_rcpm.db')

In [ ]:
ope_manager.save_to_csv('pzm_rcpm.csv')